# Assigment 7

This assigment will be graded if everything works well. I will run the script as once and everything should be done without errors and mistakes. I should be able to run your scripts in my computer and get all the results. **USE RELATIVE PATHS**. An error or exception or anything that breaks the code will means NO GRADE (0). Additionally, you are not able to modify any file handly. It also means NO GRADE (0). Comment everything you think will help others read your script. We expect 0 errors using GitHub. Everything will be graded!

**ASK EVERYTHING! WE ARE HERE TO HELP YOU!**

**GET YOUR GOOGLE API AND TOKEN. YOU WILL NEED THEM TO DO THIS TASK.**


1. Import Data from [this url](https://www.bbva.pe/content/dam/public-web/peru/documents/personas/canales-de-atencion/oficinas/Oficinas-BBVA-abiertas-23.10.20.pdf). This dataset is in PDF format. You have to convert to PandasDataFrame and keep only the BBVA offices in LIMA.
2. Use GoogleMaps API and geocode all the BBVA offices. For those offices that Google API gets no information, use internet and get the latitude and longitude handly and add them to dataset.
3. Use Google API to find the driving time (best guess) from all the group members' address and all the LIMA BBVA offices.
4. Finally, you have to give a report which offices are the most closest and furthest to every group member's address.

In [1]:
# Other packages
import urllib.request
from tqdm import tqdm_notebook as tqdm

# For sending GET requests from the API
import requests

# For saving access tokens and for file management when creating and adding to the dataset
import os

# For dealing with json responses we receive from the API
import json

# For displaying the data after
import pandas as pd
import numpy as np

# For saving the response data in CSV format
import csv

# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata

#To add wait time between requests
import time
import requests

#maps
import googlemaps

import re

In [2]:
pip install camelot-py

     ---------------------------------------- 0.0/41.0 kB ? eta -:--:--
     --------- ------------------------------ 10.2/41.0 kB ? eta -:--:--
     ---------------------------- --------- 30.7/41.0 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 41.0/41.0 kB 330.5 kB/s eta 0:00:00
  Obtaining dependency information for pdfminer.six>=20200726 from https://files.pythonhosted.org/packages/eb/9c/e46fe7502b32d7db6af6e36a9105abb93301fa1ec475b5ddcba8b35ae23a/pdfminer.six-20231228-py3-none-any.whl.metadata
  Obtaining dependency information for pypdf>=3.0.0 from https://files.pythonhosted.org/packages/12/cd/12fa4ca4262f78a984ba9cce95be824e9ce7f9dbcc1529ff4966591caef6/pypdf-4.0.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   --- ------------------------------------ 0.4/5.6 MB 8.9 MB/s eta 0:00:01
   ------ --------------------------------- 0.9/5.6 MB 11.7 MB/s eta 0:00:01
   ---------- ----------------------------- 1.4/5

In [4]:
pip install opencv-python

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/c7/ec/9dabb6a9abfdebb3c45b0cc52dec901caafef2b2c7e7d6a839ed86d81e91/opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.6 MB 656.4 kB/s eta 0:00:59
    --------------------------------------- 0.5/38.6 MB 3.9 MB/s eta 0:00:10
    --------------------------------------- 1.0/38.6 MB 5.5 MB/s eta 0:00:07
   - -------------------------------------- 1.5/38.6 MB 6.8 MB/s eta 0:00:06
   -- ------------------------------------- 2.0/38.6 MB 7.7 MB/s eta 0:00:05
   -- ------------------------------------- 2.6/38.6 MB 8.2 MB/s eta 0:00:05
   --- ------------------------------------ 3.1/38.6 MB 8.7 MB/s eta 0:00:05
   --- ------------------------------------ 3.7/38.6 MB 9.0 MB/s eta 0:00:04
   ---- -------------

In [6]:
import camelot

url = "https://www.bbva.pe/content/dam/public-web/peru/documents/personas/canales-de-atencion/oficinas/Oficinas-BBVA-abiertas-23.10.20.pdf"

# Extrae la tabla de la primera página
tables = camelot.read_pdf(url, flavor='stream', pages='2')

# Imprime la tabla
print(tables[0].df)

                       0                                             1  \
0                OFICINA                                     DIRECCION   
1             AEROPUERTO  CENTRO AÉREO COMERCIAL LOCALES 110 A Y 111 A   
2             BASE NAVAL              AV. CTRMTE.  MORA S/N BASE NAVAL   
3                              AV. ELMER FAUCETT Y ALEJANDRO BERTELLO,   
4      C.C. CANTA CALLAO                                                 
..                   ...                                           ...   
65                               ESQ. CON REPÚBLICA DOMINICANA 504-522   
66           JESUS MARIA                   AV. REP. DOMINICANA 355-359   
67  REAL PLAZA SALAVERRY      AV. GENERAL FELIPE SALAVERRY 2370, LS-04   
68            SAN FELIPE                RESIDENCIAL SAN FELIPE TDA. 23   
69                                                                       

               2          3            4  
0   DEPARTAMENTO  PROVINCIA     DISTRITO  
1           LIMA       LI

In [8]:
import camelot

url = "https://www.bbva.pe/content/dam/public-web/peru/documents/personas/canales-de-atencion/oficinas/Oficinas-BBVA-abiertas-23.10.20.pdf"

tablas_por_pagina = {}  # Diccionario para almacenar las tablas

# Itera sobre las páginas desde la 2 hasta la 9
for page_number in range(2, 6):
    try:
        # Extrae la tabla de la página actual
        tables = camelot.read_pdf(url, flavor='stream', pages=str(page_number))

        # Almacena la tabla en el diccionario
        tablas_por_pagina[page_number] = tables[0].df
    except Exception as e:
        # Maneja cualquier error que pueda ocurrir al procesar la página
        print(f"Error al procesar la página {page_number}: {str(e)}")

# Imprime las tablas almacenadas en el diccionario
for page_number, tabla in tablas_por_pagina.items():
    print(f"Tabla de la página {page_number}:")
    print(tabla)
    print("\n" + "="*50 + "\n")  # Separador visual


Tabla de la página 2:
                       0                                             1  \
0                OFICINA                                     DIRECCION   
1             AEROPUERTO  CENTRO AÉREO COMERCIAL LOCALES 110 A Y 111 A   
2             BASE NAVAL              AV. CTRMTE.  MORA S/N BASE NAVAL   
3                              AV. ELMER FAUCETT Y ALEJANDRO BERTELLO,   
4      C.C. CANTA CALLAO                                                 
..                   ...                                           ...   
65                               ESQ. CON REPÚBLICA DOMINICANA 504-522   
66           JESUS MARIA                   AV. REP. DOMINICANA 355-359   
67  REAL PLAZA SALAVERRY      AV. GENERAL FELIPE SALAVERRY 2370, LS-04   
68            SAN FELIPE                RESIDENCIAL SAN FELIPE TDA. 23   
69                                                                       

               2          3            4  
0   DEPARTAMENTO  PROVINCIA     DISTRITO  
1  

In [15]:
import pandas as pd
import camelot

url = "https://www.bbva.pe/content/dam/public-web/peru/documents/personas/canales-de-atencion/oficinas/Oficinas-BBVA-abiertas-23.10.20.pdf"

dataframes = []  # Lista para almacenar los DataFrames

# Itera sobre las páginas desde la 2 hasta la 9
for page_number in range(2, 6):
    try:
        # Extrae la tabla de la página actual
        tablas = camelot.read_pdf(url, flavor='stream', pages=str(page_number))
        
        # Almacena el DataFrame en la lista
        dataframes.append(tablas[0].df)
    except Exception as e:
        # Maneja cualquier error que pueda ocurrir al procesar la página
        print(f"Error al procesar la página {page_number}: {str(e)}")

# Concatena los DataFrames en un solo DataFrame
df_resultado = pd.concat(dataframes, ignore_index=True)

# Muestra el DataFrame resultante
print(df_resultado)

                      0                                             1  \
0               OFICINA                                     DIRECCION   
1            AEROPUERTO  CENTRO AÉREO COMERCIAL LOCALES 110 A Y 111 A   
2            BASE NAVAL              AV. CTRMTE.  MORA S/N BASE NAVAL   
3                             AV. ELMER FAUCETT Y ALEJANDRO BERTELLO,   
4     C.C. CANTA CALLAO                                                 
..                  ...                                           ...   
374                       AV. PANAMERICANA 461 CON CA. SANTO TORIBIO,   
375  SANTA ROSA SULLANA                                                 
376                                                                     
377                                             LC 6, URB. SANTA ROSA   
378              TALARA            CENTRO CIVICO 159 - BREA Y PARIÑAS   

                2          3         4  
0    DEPARTAMENTO  PROVINCIA  DISTRITO  
1            LIMA       LIMA    CALLAO  


In [17]:
tablas

<TableList n=3>